In [1]:
from src.fistanet.M5FISTANet import FISTANet
from src.fistanet.loader import DataSplit
from src.fistanet.solver import Solver
from os.path import join as pjoin
from torchsummary import summary
from datetime import datetime
import numpy as np
import torch

In [2]:
DATA_DIR = './data'
DATA_FILE_GEN = 'generated/BW_master_10000_2024-04-07-12-43-32.pkl'
DATA_FILE_SIGS = 'steinbrinker/testing_data_mvg_avg.npy'
DATA_FILE_BW = 'mit-bih/bw'
DATA_FILE_GAUSS = 'generated/gaussian_noise.npy'
DATA_FILE_BPDN = 'generated/BW_alphas-BPDN_10000_2024-04-07-12-43-32.npy'
DICT_FILE_BW = 'steinbrinker/dictionary_BW_real_data.npy'
NOISE_TYPE = 'gauss'
if NOISE_TYPE == 'bw':
    DATA_FILE_NOISE = DATA_FILE_BW
elif NOISE_TYPE == 'gauss':
    DATA_FILE_NOISE = DATA_FILE_GAUSS
DATA_SIZE = 10000
BATCH_SIZE = 1000
TVT_SPLIT = {
    'train': 80,
    'valid': 10,
    'test': 10
}

FNET_LAYER_NO = 4
FNET_FEATURE_NO = 16
LAMBDA_SP_LOSS = 1e2
LAMBDA_SYM_LOSS = 1e-1

EPOCH_NO = 3000
TEST_EPOCH = 10001
LR_DEC_AFTER = 10000
LR_DEC_EVERY = 10
START_EPOCH = 999
START_RUN = '2024-04-20-14-42-44'
LOG_INTERVAL = 4
LEARNING_RATE = 1e-3



# DATA_FILE_GEN = 'generated/BW_master_7999-8000_2024-04-07-12-43-32.pkl'
# DATA_SIZE = 2
# BATCH_SIZE = 1
# TVT_SPLIT = {
#     'train': 50,
#     'valid': 50,
#     'test': 0
# }
# FNET_LAYER_NO = 4
# FNET_FEATURE_NO = 16
# LEARNING_RATE = 1e-3
# LAMBDA_SP_LOSS = 0.5

In [3]:
trn_ldr, val_ldr, tst_ldr = DataSplit(DATA_DIR, NOISE_TYPE, DATA_FILE_GEN, DATA_FILE_SIGS, DATA_FILE_NOISE, DATA_FILE_BPDN, TVT_SPLIT, BATCH_SIZE)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
Psi = np.load(pjoin(DATA_DIR, DICT_FILE_BW))
Psi = torch.from_numpy(Psi)
Psi = Psi.clone().detach().to(device=device)

In [6]:
fista_net = FISTANet(FNET_LAYER_NO, FNET_FEATURE_NO)
fista_net = fista_net.to(device)# define arguments of fista_net


In [7]:
# summary(fista_net, input_size=(1, 64, 298), device=str(device))
print('Total number of parameters fista net:',
          sum(p.numel() for p in fista_net.parameters()))

Total number of parameters fista net: 18871


In [8]:
dt = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
if START_EPOCH:
    dt = START_RUN
args = {
    'model_name': 'FISTANet',
    'num_epochs': EPOCH_NO,
    'lr': LEARNING_RATE,
    'data_dir': DATA_DIR,
    'save_path': f'./runs/{dt}',
    'start_epoch': START_EPOCH,
    'start_run': START_RUN,
    'multi_gpu': False,
    'device': device,
    'log_interval': LOG_INTERVAL,
    'test_epoch': TEST_EPOCH,
    'lr_dec_after': LR_DEC_AFTER,
    'lr_dec_every': LR_DEC_EVERY,
    'lambda_sp_loss': LAMBDA_SP_LOSS,
    'lambda_sym_loss': LAMBDA_SYM_LOSS
}

In [9]:
solver = Solver(fista_net, Psi, trn_ldr, val_ldr, BATCH_SIZE, args, tst_ldr)

In [10]:
solver.train()

Training epoch 1000...

Train Epoch: 1000 [0/8000 (0%)]	Batch Loss: 1174.782244	Learning Rate (w_theta): 0.001000	 TIME:6.6s
				Disc: 1.804112		Sym: 21.282454		Spars: 1151.695679
	 TVw: -0.531766 | TVb: -2.043251 | GSw: -0.234959 | GSb: 0.065070 | TSUw: 0.464946 | TSUb: 0.034764

Train Epoch: 1000 [4000/8000 (50%)]	Batch Loss: 1232.489828	Learning Rate (w_theta): 0.001000	 TIME:8.1s
				Disc: 2.106847		Sym: 24.725266		Spars: 1205.657715
	 TVw: -0.531744 | TVb: -2.043251 | GSw: -0.234959 | GSb: 0.065070 | TSUw: 0.464946 | TSUb: 0.034764
Validating epoch 1000...
-------------------------------------------
Epoch statistics:
Average training loss: 1222.9144150249917
Average validation loss: 234.96653688042127
Training epoch 1001...

Train Epoch: 1001 [0/8000 (0%)]	Batch Loss: 1204.635356	Learning Rate (w_theta): 0.001000	 TIME:11.0s
				Disc: 1.883180		Sym: 23.192606		Spars: 1179.559570
	 TVw: -0.531721 | TVb: -2.043251 | GSw: -0.234959 | GSb: 0.065070 | TSUw: 0.464946 | TSUb: 0.034764

T

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 20000000 bytes.